- Họ tên SV: Trần Minh Tiến
- MSSV: 23521587
- Yêu cầu: hoàn thành các phần code thiếu: #code here

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
#importing necessary libraries
import numpy as np
import pandas as pd
# thêm thư viện TfidfVectorizer
#core here
from sklearn.feature_extraction.text import TfidfVectorizer
#thêm thư viện linear_kernel
#core here
from sklearn.metrics.pairwise import linear_kernel


In [81]:
#Load dữ liệu movies_metadata.csv
#code here
# movies = ?

movies = pd.read_csv('movies/movies_metadata.csv', low_memory=False)


In [82]:
#xuất ra overview của 5 bộ phim đầu tiên
#code here

print(movies[['overview']].head(5))

                                            overview
0  Led by Woody, Andy's toys live happily in his ...
1  When siblings Judy and Peter discover an encha...
2  A family wedding reignites the ancient feud be...
3  Cheated on, mistreated and stepped on, the wom...
4  Just when George Banks has recovered from his ...


In [83]:
# viết hàm tiền xử lý dữ liệu trên cột overview
#  - chuyển về từ thường
#  - Xóa dấu câu, ký tự đặc biệt
#  - xóa stopword
#  - tách từ
#  ...
#code here

import re

def preprocess_overview(text):
    text = str(text).lower()
    return text

# Áp dụng tiền xử lý
movies['overview'] = movies['overview'].apply(preprocess_overview)
movies['overview'] = movies['overview'].fillna('')

movies['overview'].head(5)

0    led by woody, andy's toys live happily in his ...
1    when siblings judy and peter discover an encha...
2    a family wedding reignites the ancient feud be...
3    cheated on, mistreated and stepped on, the wom...
4    just when george banks has recovered from his ...
Name: overview, dtype: object

In [84]:
#sử dụng TfidfVectorizer để chuyển cột overview của các bộ phim về tf-idf và lưu vào biến overview_matrix
# code here

tfidf = TfidfVectorizer(stop_words='english')
overview_matrix = tfidf.fit_transform(movies['overview'])

# Lưu vectorizer nếu cần sau này
# import pickle
# pickle.dump(tfidf, open('tfidf_overview.pkl', 'wb'))

overview_matrix.shape

(45466, 75827)

In [85]:
# in ra shape của overview_matrix nếu = (45466, 75827) là đúng.
#code here

print('overview_matrix.shape =', overview_matrix.shape)

overview_matrix.shape = (45466, 75827)


In [86]:
# Tính toán cosine similarity giữa các bộ phim bằng linear_kernel
cosine_sim = linear_kernel(overview_matrix, overview_matrix)

# Ví dụ: cosine_sim[i] là vector similarity của phim i với tất cả phim khác

MemoryError: Unable to allocate 15.4 GiB for an array with shape (45466, 45466) and data type float64

In [ ]:
#đánh index cho các bộ phim bằng  pd.Series() và lưu trong biến mapping
#code here

mapping = pd.Series(movies.index, index=movies['title'].astype(str)).drop_duplicates()
mapping[:5]

In [ ]:
#viết hàm nhận vào index của bộ phim trả về tên của bộ phim
#code here

def get_title_from_index(index):
    try:
        return movies.loc[index, 'title']
    except Exception:
        return None

# Hoặc lấy index từ title:
def get_index_from_title(title):
    try:
        return int(mapping[title])
    except Exception:
        return None

In [ ]:
#viết hàm trả và top 10 bộ phim sẽ đem đi khuyến nghị cho người dùng U. Người dung U đ

def recommend_movies_for_user(title_watched, top_n=10):
    # Lấy index phim đã xem
    idx = get_index_from_title(title_watched)
    if idx is None:
        raise ValueError(f'Không tìm thấy phim: {title_watched}')

    # Lấy danh sách tuples (index, similarity)
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sắp xếp theo similarity giảm dần
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Bỏ qua chính phim đã xem (xứng với idx ở vị trí đầu)
    sim_scores = [s for s in sim_scores if s[0] != idx]

    # Lấy top N
    top_indices = [i for i, _ in sim_scores[:top_n]]

    # Trả về DataFrame các phim khuyến nghị
    return movies.loc[top_indices, ['title', 'overview']].reset_index(drop=True)

# Ví dụ sử dụng
watched = 'Father of the Bride Part II'
print(f'Phim người dùng đã xem: {watched}')

display(recommend_movies_for_user(watched, top_n=10))Top 10 phim được khuyến nghị:')